In [4]:
import verde as vd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pyproj
from shapely import geometry


%matplotlib widget

gdb = '/home/ggrl/geodatabase/'

In [2]:
m_rparaim = pd.read_csv(gdb+'geof/recortado/m_rparaim')

In [5]:
g_rparaim = pd.read_csv(gdb+'geof/recortado/g_rparaim')

In [4]:
g_rparaim.sort_values('LATITUDE')

,X,Y,MDT,KPERC,eU,eth,CTCOR,LONGITUDE,LATITUDE
37611,523111.29,8784013.32,484.62,0.37,1.27,7.40,695.04,-44.788444,-10.999992
51744,533602.25,8784004.71,452.52,0.48,0.47,7.41,625.33,-44.692412,-10.999988
65711,543613.15,8783993.73,452.72,0.22,0.41,5.58,436.27,-44.600776,-10.999980
62911,541130.10,8783998.16,443.86,0.15,0.49,3.56,310.58,-44.623504,-10.999969
32024,519113.19,8784018.60,516.49,0.85,1.52,4.85,726.11,-44.825041,-10.999967
...,...,...,...,...,...,...,...,...,...
17444,509093.55,8839297.36,426.09,0.12,1.28,8.20,668.69,-44.916896,-10.500057
78993,552606.04,8839259.22,490.75,0.60,1.35,15.88,1256.65,-44.519250,-10.500049
58049,537617.81,8839279.49,437.43,0.46,0.87,8.99,759.14,-44.656221,-10.500044
43343,527623.79,8839292.72,441.65,0.27,1.92,14.35,1145.37,-44.747553,-10.500010


In [5]:
m_rparaim.sort_values('MAGIGRF')

,X,Y,MDT,MAGIGRF,LONGITUDE,LATITUDE
578650,537614.77,8824535.33,638.68,-647.075,-44.656101,-10.633392
578651,537614.87,8824543.45,638.79,-647.071,-44.656100,-10.633318
578649,537614.67,8824527.18,639.01,-646.876,-44.656101,-10.633465
578652,537614.97,8824551.58,638.89,-646.872,-44.656099,-10.633244
578653,537615.07,8824559.71,639.00,-646.490,-44.656098,-10.633171
...,...,...,...,...,...,...
582452,538127.30,8823831.81,632.37,327.884,-44.651408,-10.639749
582451,538127.60,8823839.42,632.71,328.196,-44.651405,-10.639680
582448,538128.51,8823862.25,633.72,328.211,-44.651397,-10.639474
582450,538127.90,8823847.03,633.04,328.367,-44.651402,-10.639611


In [8]:
coordinates = (g_rparaim.X.values, g_rparaim.Y.values)
coordinates

(array([500020.25, 500103.83, 500187.24, ..., 554618.94, 554618.52,
        554617.97]),
 array([8786155.43, 8786158.77, 8786161.64, ..., 8839028.43, 8839107.8 ,
        8839187.24]))

In [7]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c    = g_rparaim.CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
g_rparaim['geometry'] = [geometry.Point(x,y) for x, y in zip(g_rparaim['LONGITUDE'], g_rparaim['LATITUDE'])]

In [9]:
# Block Reduction
reducer_median = vd.BlockReduce(np.median, spacing= 500)
b_coords, b_CTCOR = reducer_median.filter(coordinates, g_rparaim.CTCOR)

plt.figure(figsize=(11,11))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
reducer_mean = vd.BlockReduce(np.mean, spacing= 499)
b_coords, b_CTCOR = reducer_mean.filter(coordinates, g_rparaim.CTCOR)

plt.figure(figsize=(11,11))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
reducer_mean = vd.BlockReduce(np.mean, spacing= 1000)
b_coords, b_CTCOR = reducer_mean.filter(coordinates, g_rparaim.CTCOR)

In [14]:
spline = vd.Spline()

In [15]:
spline.fit(b_coords, b_CTCOR)

Spline()

In [16]:
predicted = spline.predict(coordinates)

In [17]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='magma',
            s=1)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
residuals = g_rparaim.CTCOR - predicted

In [19]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 200,pixel_register=True)

In [25]:
region

(500005.3, 554693.86, 8783985.23, 8839303.77)

In [26]:
grid_CTCOR = spline.predict(grid_coords)

In [27]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_CTCOR,
            cmap='magma',
            s=2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
grid = spline.grid(spacing=200, data_names=['CTCOR'])
grid

<xarray.Dataset>
Dimensions:   (easting: 271, northing: 273)
Coordinates:
  * easting   (easting) float64 5.003e+05 5.005e+05 ... 5.542e+05 5.544e+05
  * northing  (northing) float64 8.784e+06 8.785e+06 ... 8.839e+06 8.839e+06
Data variables:
    CTCOR     (northing, easting) float64 1.618e+03 1.614e+03 ... 1.131e+03
Attributes:
    metadata:  Generated by Spline()

In [33]:
'''
vmin= grid.CTCOR.quantile([0.25,0.5,0.75])
print(vmin)
'''
grid.CTCOR.plot(figsize=(12,12),
               cmap='hsv',vmin=600, vmax=2000)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500225.1398596865, 554503.805909544, 8784373.113641337, 8838919.583069772)

In [35]:
grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid)
grid

grid.CTCOR.plot(figsize=(12,12),cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500225.1398596865, 554503.805909544, 8784373.113641337, 8838919.583069772)

In [6]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.median, spacing=2000)),
    ('spline', vd.Spline())
])

In [47]:
g_rparaim.head()

,X,Y,MDT,KPERC,eU,eth,CTCOR,LONGITUDE,LATITUDE,geometry
0,500020.25,8786155.43,477.68,0.35,1.25,16.12,1182.85,-44.999815,-10.980692,POINT (-44.99981462 -10.9806923)
1,500103.83,8786158.77,476.83,0.63,1.24,14.00,1143.89,-44.999050,-10.980662,POINT (-44.99904958 -10.98066207)
2,500187.24,8786161.64,476.58,1.02,1.38,12.02,1165.74,-44.998286,-10.980636,POINT (-44.99828615 -10.9806361)
3,500270.50,8786163.59,470.91,1.41,1.51,11.31,1257.68,-44.997524,-10.980618,POINT (-44.99752405 -10.98061848)
4,500353.54,8786165.69,464.63,1.71,1.65,11.14,1357.00,-44.996764,-10.980599,POINT (-44.99676395 -10.98059948)


In [9]:
chain.fit(coordinates, g_rparaim.KPERC)

grid = chain.grid(spacing=400, data_names=['KPERC'])
grid_mask = vd.distance_mask(coordinates, maxdist=2000, grid=grid)

In [10]:
grid_mask

<xarray.Dataset>
Dimensions:   (easting: 138, northing: 139)
Coordinates:
  * easting   (easting) float64 5e+05 5.004e+05 ... 5.543e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    KPERC     (northing, easting) float64 1.391 1.288 1.177 ... 0.6268 0.6577
Attributes:
    metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...

In [11]:
grid_mask.KPERC.plot(figsize=(8,8), cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499805.70671532844, 554893.4532846715, 8783784.800507247, 8839504.199492753)

Model Validation

In [83]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR,
                                 test_size = 0.1)

In [84]:
train

((array([510111.05, 511611.91, 540119.8 , ..., 537636.97, 521580.05,
         506097.05]),
  array([8803812.52, 8837463.09, 8830925.37, ..., 8792793.09, 8810432.26,
         8820906.92])),
 (array([1430.24,  719.39, 1082.28, ...,  687.89, 1114.03, 1300.79]),),
 (None,))

In [85]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.872, 557428.288, 8781219.303000001, 8842069.696999999)

In [86]:
chain_CTCOR.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function median at 0x7f6a67efaca0>,
                          spacing=1000)),
             ('spline', Spline())])

In [87]:
chain_CTCOR.score(*test)

0.8171088537170001

In [108]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR, test_size=0.1, spacing=1000)

In [109]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=3)
plt.plot(test[0][0], test[0][1], '.r', markersize=3)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.872, 557428.288, 8781219.303000001, 8842069.696999999)

In [110]:
chain_CTCOR.fit(*train)

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function median at 0x7f6a67efaca0>,
                          spacing=1000)),
             ('spline', Spline())])

In [111]:
chain_CTCOR.score(*test)

0.6854460461252302

In [94]:
cv     = vd.BlockKFold(spacing=1000,
                  n_splits=50,
                  shuffle=True)

In [95]:
scores = vd.cross_val_score(chain_CTCOR,
                            coordinates,
                            g_rparaim.CTCOR,
                            cv=cv)

In [96]:
scores

array([0.76600745, 0.65691566, 0.79872783, 0.73130954, 0.73478961,
       0.77517123, 0.63237133, 0.75020688, 0.67563968, 0.7580242 ,
       0.79296449, 0.68999138, 0.7871542 , 0.72674868, 0.77997202,
       0.69916478, 0.62143947, 0.72640941, 0.6324465 , 0.65074986,
       0.73982833, 0.67945018, 0.74135313, 0.67643012, 0.79161596,
       0.69180662, 0.68290468, 0.6929264 , 0.75604804, 0.83531106,
       0.72031158, 0.73842824, 0.75775224, 0.7974437 , 0.81237241,
       0.70312159, 0.63339506, 0.74506223, 0.64092632, 0.73596647,
       0.79427739, 0.76603196, 0.76096846, 0.61802663, 0.73571703,
       0.76286133, 0.68643094, 0.60970192, 0.7428681 , 0.81303388])

In [97]:
plt.figure()
plt.hist(scores, bins ='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 7.,  2., 10.,  8., 12.,  8.,  3.]),
 array([0.60970192, 0.6419318 , 0.67416168, 0.70639155, 0.73862143,
        0.7708513 , 0.80308118, 0.83531106]),
 <BarContainer object of 7 artists>)

In [100]:
geof = CTCOR_grid.to_dataframe()
geof.reset_index(inplace=True)

In [101]:
geof

,easting,northing,KPERC
0,500005.30,8.783985e+06,1659.578181
1,500005.30,8.784185e+06,1634.598105
2,500005.30,8.784385e+06,1599.014773
3,500005.30,8.784584e+06,1551.296070
4,500005.30,8.784784e+06,1492.718534
...,...,...,...
76167,554693.86,8.838505e+06,1171.607396
76168,554693.86,8.838705e+06,1173.625707
76169,554693.86,8.838904e+06,1174.655027
76170,554693.86,8.839104e+06,1174.081106
